Zad.1
Porównaj zapisywanie i odczytywanie kolekcji (100, 10000, 100 000 elementów) za pomocą trzech technik: modułu pickle, parquet i xlsx.

In [90]:
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
from pickle import dump, load
from tempfile import mktemp
from os import unlink
import time
from fastparquet import ParquetFile


keys100 = list(range(0,100))
for i in range(len(keys100)):
    keys100[i] = f'element{i}'
    
keys10000 = list(range(0,10000))
for i in range(len(keys10000)):
    keys10000[i] = f'element{i}'
    
keys100000 = list(range(0,100000))
for i in range(len(keys100000)):
    keys100000[i] = f'element{i}'

dict100 = dict.fromkeys(keys100, list(range(1,3)))
dict10000 = dict.fromkeys(keys10000, list(range(1,3)))
dict100000 = dict.fromkeys(keys100000, list(range(1,3)))

def Save_Load_Pickle(collection, file):
    with open(file, 'wb') as f:
        start = time.time()
        dump(collection, f)
        end = time.time()

        print(f'Czas operacji zapisu dla kolekcji o długości {len(collection)} w module pickle: ok. {round(end - start, 10)} s')

        f.flush()

    with open(file, 'rb') as f:
        sumOf = 0
        start = time.time()
        temp_col = load(f)
        end = time.time()

        print(f'Czas operacji odczytu dla kolekcji o długości {len(collection)} w module pickle: ok. {round(end - start, 10)} s\n')

temp_file = mktemp()
Save_Load_Pickle(dict100, temp_file)
unlink(temp_file)

temp_file = mktemp()
Save_Load_Pickle(dict10000, temp_file)
unlink(temp_file)

temp_file = mktemp()
Save_Load_Pickle(dict100000, temp_file)
unlink(temp_file)

def Save_Load_Parquet(collection, file):
    df = pd.DataFrame.from_dict(collection)
    
    start = time.time()
    df.to_parquet(file, compression='GZIP')
    end = time.time()

    print(f'Czas operacji zapisu dla kolekcji o długości {len(collection)} w module parquet: ok. {round(end - start, 10)} s')

    start = time.time()
    pf = ParquetFile(file)
    end = time.time()

    print(f'Czas operacji odczytu dla kolekcji o długości {len(collection)} w module parquet: ok. {round(end - start, 10)} s\n')


temp_file = mktemp()
Save_Load_Parquet(dict100, temp_file)
unlink(temp_file)

temp_file = mktemp()
Save_Load_Parquet(dict10000, temp_file)
unlink(temp_file)

temp_file = mktemp()
Save_Load_Parquet(dict100000, temp_file)
unlink(temp_file)

def Save_Load_Xlsx(collection, file):
    try:
        df = pd.DataFrame.from_dict(collection)
        with open(file, 'wb') as f:
            wb = Workbook()
            ws = wb.active

            for r in dataframe_to_rows(df, index=True, header=True):
                ws.append(r)

            for cell in ws['A'] + ws[1]:
                cell.style = 'Pandas'

            start = time.time()
            wb.save(f.name)
            end = time.time()

            print(f'Czas operacji zapisu dla kolekcji o długości {len(collection)} w module xlsx: ok. {round(end - start, 10)} s')

            f.flush()

        with open(file, 'rb') as f:
            start = time.time()
            wb = load_workbook(f.name)
            end = time.time()

            print(f'Czas operacji odczytu dla kolekcji o długości {len(collection)} w module xlsx: ok. {round(end - start, 10)} s\n')
    except Exception as e:
        print(f'\nWystąpił błąd podczas zapisu pliku {len(collection)} elementowego: {e}!')
        print(f'Wiadomość {e} oznacza, że nie można zapisać pliku o tak dużej ilości kolumn')
        
temp_file = mktemp(suffix='.xlsx')   
Save_Load_Xlsx(dict100, temp_file)
unlink(temp_file)

temp_file = mktemp(suffix='.xlsx')   
Save_Load_Xlsx(dict10000, temp_file)
unlink(temp_file)

temp_file = mktemp(suffix='.xlsx')   
Save_Load_Xlsx(dict100000, temp_file)
unlink(temp_file)
    

print('\n Podsumowanie:\n')
print('Pickle [odczyt szybszy niż zapis]\nMożemy zauważyć, że najkrótsze czasy odczytu i zapisu słowników 100, 10000 i 100000 elementowych\nwystępują dla modułu pickle, jest to związane z faktem, że ów modół zapisuje mniej skomplikowane dane do pliku (słownik),\nlecz jest najszybszy. Oczywiście nie umożliwia nam on zapisywania struktur bardziej złożonych jak: dataframe lub plik \".xlsx\".\n\nParquet [odczyt znacząco szybszy niż zapis]\nKolejny moduł - parquet umożliwia nam zapis i odczyt struktury dataframe, zapis zajmuje zdecydowanie dłużej czasu od pickle i \nxlsx, lecz jest bardzo wygodny do działań na obiektach dataframe. Umożliwia nam również kompresje pliku.\n\nXlsx [zapis szybszy niż odczyt]\nOstatni moduł - xlsx pozwala na zapis i odczyt plików \".xslx\", jest bardzo szybki w swoich działaniach, lecz nie pozwala nam\nzapisywać ogromnych danych ze względu na ograniczenie kolumn w pliku \".xslx\" - Invalid column index 100001.\n\nKażdy z powyższych modułów jest użyteczny na inny sposób i służy do innych czynności.')

Czas operacji zapisu dla kolekcji o długości 100 w module pickle: ok. 0.0 s
Czas operacji odczytu dla kolekcji o długości 100 w module pickle: ok. 0.0 s
Czas operacji zapisu dla kolekcji o długości 10000 w module pickle: ok. 0.0010027885 s
Czas operacji odczytu dla kolekcji o długości 10000 w module pickle: ok. 0.0009965897 s
Czas operacji zapisu dla kolekcji o długości 100000 w module pickle: ok. 0.0141186714 s
Czas operacji odczytu dla kolekcji o długości 100000 w module pickle: ok. 0.0156621933 s
Czas operacji zapisu dla kolekcji o długości 100 w module parquet: ok. 0.0868506432 s
Czas operacji odczytu dla kolekcji o długości 100 w module parquet: ok. 0.0 s
Czas operacji zapisu dla kolekcji o długości 10000 w module parquet: ok. 8.3829042912 s
Czas operacji odczytu dla kolekcji o długości 10000 w module parquet: ok. 0.1428835392 s
Czas operacji zapisu dla kolekcji o długości 100000 w module parquet: ok. 79.3076577187 s
Czas operacji odczytu dla kolekcji o długości 100000 w module pa

Zad. 2
Zbadaj przyspieszenie związane z zastosowaniem pamięci podręcznej na wybranych przykładzie funkcji rekurencyjnej 
(np. ciągu Fibonacciego).

In [18]:
from functools import lru_cache
import time

@lru_cache(maxsize=None)
def fib_mem(n: int):
    if n < 2:
        return n
    return fib_mem(n-1) + fib_mem(n-2)

def fib_reg(n: int):
    if n <= 1:
        return n;
    return fib_reg(n-1) + fib_reg(n-2);

#Liczba pierwszych znaków ciągu do wypisania
x = 30
def fibonacciCheck(x: int):
    start = time.time_ns()
    print([fib_mem(n) for n in range(x)])
    end = time.time_ns()
    print(f'Wypisanie pierwszych {x} znaków ciągu Fibonacciego przy użyciu pamięci podręcznej zajęło: ok. {end - start} ns')

    start = time.time_ns()
    print([fib_reg(n) for n in range(x)])
    end = time.time_ns()
    print(f'Wypisanie pierwszych {x} znaków ciągu Fibonacciego bez użycia pamięci podręcznej zajęło: ok. {end - start} ns\n')

fibonacciCheck(10)
fibonacciCheck(20)
fibonacciCheck(30)
fibonacciCheck(35)        # Liczby powyżej 30 liczby Fibonacchiego obliczane są bardzo długo przez fib_reg


print('\n Możemy zauważyć ogromną różnicę w wydajności, przy zastosowaniu pamięci podręcznej')

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
Wypisanie pierwszych 10 znaków ciągu Fibonacciego przy użyciu pamięci podręcznej zajęło: ok. 0 ns
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
Wypisanie pierwszych 10 znaków ciągu Fibonacciego bez użycia pamięci podręcznej zajęło: ok. 0 ns

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]
Wypisanie pierwszych 20 znaków ciągu Fibonacciego przy użyciu pamięci podręcznej zajęło: ok. 0 ns
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]
Wypisanie pierwszych 20 znaków ciągu Fibonacciego bez użycia pamięci podręcznej zajęło: ok. 6981700 ns

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229]
Wypisanie pierwszych 30 znaków ciągu Fibonacciego przy użyciu pamięci podręcznej zajęło: ok. 0 ns
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 286

Zad. 3
Napisz program tworzący bazę danych z interfejsem konsolowym. Wymagane są następujące operacje, dodanie wiersza, usunięcie wiersza, zmiana pola wiersza, wyświetlenie opcji. Menu można zorganizować jako odczytywanie parametrów zwróconych przez funkcję input lub z pomocą komend (łatwiejszy sposób).

In [94]:
from sqlalchemy import create_engine, Column, String, Integer, Boolean, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from contextlib import contextmanager
from tempfile import mktemp
from platform import system

temp_db = mktemp(suffix='.sqlite')

conn_uri_handler = {
    'Windows': f'sqlite:///{temp_db}',
    'Darwin': f'sqlite:////{temp_db}',
    'Linux': f'sqlite:////{temp_db}',
    'Java': f'sqlite:////{temp_db}'
}

# w Windows dodaje dodatkowo /
engine = create_engine(conn_uri_handler[system()])

Base = declarative_base(bind=engine)

class Product(Base):
    __tablename__ = 'products'
    id=Column(Integer, primary_key=True)
    title=Column('title', String(32))
    in_stock=Column('in_stock', Boolean)
    quantity=Column('quantity', Integer)


Base.metadata.create_all()

Session = sessionmaker(bind=engine)

@contextmanager
def create_session():
    session = Session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()

def provide_session(func):
    def wrapper(*args, **kwargs):
        try:
            with create_session() as session:
                args = (*args, session) if args else (session,) # dodanie parametru session do listy
                return func(*args, **kwargs)
        except Exception as e:
            print(f'Error found: {e}')

        return None

    return wrapper

@provide_session
def populate(session):
    global in_id, in_title, in_in_stock, in_quantity
    session.add(Product(id=1, title='Laptop Sony', in_stock=True, quantity=19))
    session.add(Product(id=2, title='Laptop Lenovo', in_stock=True, quantity=6))
    session.add(Product(id=3, title='Laptop Lenovo 2', in_stock=False, quantity=0))
    session.add(Product(id=4, title='Laptop Dell', in_stock=False, quantity=0))
    session.add(Product(id=5, title='Laptop HP', in_stock=True, quantity=100))
    session.add(Product(id=6, title='Laptop HP 2', in_stock=True, quantity=67))
    session.add(Product(id=7, title='Laptop HP 3', in_stock=False, quantity=0))
    
@provide_session
def add(session):
    next_id = session.query(Product).order_by('id').all()[-1].id + 1
    
    global in_title, in_in_stock, in_quantity
    session.add(Product(id=next_id, title=in_title, in_stock=in_in_stock, quantity=in_quantity))
    
@provide_session
def delete(session):
    global in_id
    obj = session.query(Product).filter(Product.id==in_id).first()
    session.delete(obj)
    session.commit()

@provide_session
def update(session):
    global in_id, in_title, in_in_stock, in_quantity
    
    if in_title != "":
        obj = session.query(Product).filter(Product.id==in_id).update({'title': in_title}, synchronize_session="fetch")
    if in_in_stock != "":
        obj = session.query(Product).filter(Product.id==in_id).update({'in_stock': in_in_stock}, synchronize_session="fetch")
    if in_quantity != "":
        obj = session.query(Product).filter(Product.id==in_id).update({'quantity': in_quantity}, synchronize_session="fetch")
    
    session.commit()
    
@provide_session
def query(session):
    available_products = session.query(Product).filter(Product.in_stock == True).all()
    print('\nDostępne produkty:')
    for product in available_products:
        print(f'{product.id}\t{product.title}\t{product.quantity}')

    available_products = session.query(Product).filter(Product.in_stock == False).all()
    print('Niedostępne produkty:')
    for product in available_products:
        print(f'{product.id}\t{product.title}\t{product.quantity}')
    print("\n")


# ID w bazie danych ustalane jest automatycznie (nie można go zmieniać [chyba, że ktoś usunie i doda rekord]), ustawiane jest na 
# najwyższe w bazie danych + 1
back = True
# Na początku 'zapełniam' bazę danych przykładowymi rekordami
populate()

# Komunikacja z użytkownikiem:

# Zmienne globalne, wpisywane (wybierane) przez użytkownika, służące do sterowania funkcjami add, delete, update
while(back):
    in_id = None
    in_title = None
    in_in_stock = None
    in_quantity = None
    in_to_change = None

    print('Aby wybrać jedną opcje wpisuj numerek jej odpowiadający.')

    x = input('Wybierz czynność, którą chcesz wykonać:\n1. Dodaj obiekt\n2. Usuń obiekt\n3. Edytuj obiekt\n4. Wyświetl tabelę\n5. Wyjście\n')
    try:    
        x = int(x)
        if x<1 or x>5:
            raise Exception
        if(x==1):
            print('\nWpisz dane nowego rekordu (ID ustalane jest automatycznie):')
            in_title = input('Wpisz nazwę przedmiotu:\n')

            in_in_stock = input('Ustal czy jest na stanie:\n1. Tak\n2. Nie\n')
            if(in_in_stock == '1'):
                in_in_stock = True
            elif(in_in_stock == '2'):
                in_in_stock = False
            else:
                raise Exception
            in_quantity = int(input('Ustal jaka ilość przedmiotu jest na stanie (tylko wartości numeryczne):\n'))
            add()

        elif(x==2):
            query()
            in_id = int(input('\nWybierz, który obiekt chcesz usunąć, wybierając jego numer z lewej strony:'))
            delete()

        elif(x==3):
            query()
            in_id = int(input('\nWybierz, który obiekt chcesz edytować, wybierając jego numer z lewej strony:'))
            in_title = input('Wybierz nową nazwę przedmiotu (jeżeli nie chcesz zmieniać pozostaw puste):\n')
            in_in_stock = input('Wybierz nowy stan(jeżeli nie chcesz zmieniać pozostaw puste):\n1. Tak\n2. Nie\n')
            if(in_in_stock == '1'):
                in_in_stock = True
            elif(in_in_stock == '2'):
                in_in_stock = False
            else:
                raise Exception
            in_quantity = int(input('Wybierz nową ilośc przedmiotu na stanie (jeżeli nie chcesz zmieniać pozostaw puste):\n'))
            update()
        elif(x==4):
            query()
        else:
            back = False
    except:
        print('To nie jest poprawny numer!')

# Oczywiście aplikacja mogłaby być bardziej przyjazna np. wracać do miejsca gdzie wpisaliśmy złą liczbę, zakładać, że na 
# stanie przedmiotów jest 0, gdy są niedostępne itd. Takie rozwiązanie oczywiście jest lepsze, lecz uważam, że na potrzeby
# Zadania nie jest to aż tak istotne, obsługa błędów na dość dobrym poziomie znajduje się w programie. Każda konwersja int(),
# jest wyłapywana w zewnętrznym except'ie, pętla jest kontynuowana 

Aby wybrać jedną opcje wpisuj numerek jej odpowiadający.
Wybierz czynność, którą chcesz wykonać:
1. Dodaj obiekt
2. Usuń obiekt
3. Edytuj obiekt
4. Wyświetl tabelę
5. Wyjście
3

Dostępne produkty:
1	Laptop Sony	19
2	Laptop Lenovo	6
5	Laptop HP	100
6	Laptop HP 2	67
Niedostępne produkty:
3	Laptop Lenovo 2	0
4	Laptop Dell	0
7	Laptop HP 3	0



Wybierz, który obiekt chcesz edytować, wybierając jego numer z lewej strony:2
Wybierz nową nazwę przedmiotu (jeżeli nie chcesz zmieniać pozostaw puste):

Wybierz nowy stan(jeżeli nie chcesz zmieniać pozostaw puste):
1. Tak
2. Nie
1
Wybierz nową ilośc przedmiotu na stanie (jeżeli nie chcesz zmieniać pozostaw puste):
12
Aby wybrać jedną opcje wpisuj numerek jej odpowiadający.
Wybierz czynność, którą chcesz wykonać:
1. Dodaj obiekt
2. Usuń obiekt
3. Edytuj obiekt
4. Wyświetl tabelę
5. Wyjście
4

Dostępne produkty:
1	Laptop Sony	19
2	Laptop Lenovo	12
5	Laptop HP	100
6	Laptop HP 2	67
Niedostępne produkty:
3	Laptop Lenovo 2	0
4	Laptop Dell	0
7	Laptop HP 3	